# WEBTOON ORIGINALS

This notebook serves to `GET` data from the Webtoon API on RapidAPI and convert the `JSON` repsonse to `CSV` format.

[Webtoon API](https://rapidapi.com/apidojo/api/webtoon/)

## Requirements

**Libraries**

- requests

**API Key**

A RapidAPI key will be required to get data.

If you do not have a RapidAPI account, you can get a key at [Webtoon API on RapidAPI](https://rapidapi.com/apidojo/api/webtoon/) by going to "Pricing", subscribing to the Basic plan, and creating an account.

If you already have an account, subscribe to the API in order to use it.

In [1]:
from requests import request
from csv import writer

Function that makes the API call. Returns a JSON object.

In [2]:
def api_call(lang):
   rapid_api_key = "df49ecb235msh2daa0d8ec5f5918p1bc1dcjsn1c7de24bf1c9" # rapid api key here
   
   url = "https://webtoon.p.rapidapi.com/originals/titles/list"
   
   querystring = {"language": lang}
   
   headers = {
      "x-rapidapi-host": "webtoon.p.rapidapi.com",
      "x-rapidapi-key": rapid_api_key
   }
   
   response = request("GET", url, headers = headers, params = querystring)
   
   titles = response.json()["message"]["result"]["titleList"]["titles"]
   
   return(titles)

Function that makes the csv. Does a bit of cleaning and changes the header to be more readable.

In [3]:
def make_data(titles, lang):
   dataset = open("webtoon_originals_" + lang + ".csv", "w", newline = "", encoding = "utf-8")
   
   csv_writer = writer(dataset, delimiter = ",")
   
   title_keys = ["titleNo", "title", "representGenre", "titleAuthorList", "weekday",
                 "totalServiceEpisodeCount", "favoriteCount", "starScoreAverage", "readCount",
                 "likeitCount", "restTerminationStatus", "dailyPassTitle", "synopsis"
                ]
   header = ["title_id", "title", "genre", "authors", "weekdays", "length", "subscribers",
             "rating", "views", "likes", "status", "daily_pass", "synopsis"
            ]
   
   csv_writer.writerow(header)
   
   for title in titles:
      title_vals = []
      for k in title_keys:
         auth_list = []
         wkdy_list = []
         if k == "titleAuthorList":
            for auth in title[k]:
               auth_list.append(auth["authorName"])
            auth_str = ",".join(auth_list)
            title_vals.append(auth_str)
         elif k == "weekday":
            for wkdy in title[k]:
               wkdy_list.append(wkdy)
            wkdy_str = ",".join(wkdy_list)
            title_vals.append(wkdy_str)
         elif k == "restTerminationStatus":
            if title[k] == "SERIES":
               title_vals.append("ONGOING")
            elif title[k] == "TERMINATION":
               title_vals.append("COMPLETED")
            elif title[k] == "REST":
               title_vals.append("HIATUS")
            else:
               title_vals.append(title[k])
         else:
            title_vals.append(title[k])
      csv_writer.writerow(title_vals)
   
   dataset.close()

Iterates though all the languages and makes the API call and csv for each language.

In [4]:
langs = ["en", "zh-hant", "th", "id", "es", "fr", "de"]

for lang in langs:
   titles = api_call(lang)
   make_data(titles, lang)

In [1]:
import pandas as pd

# Percorsi dei file CSV per le lingue
file_paths = {
    "en": "./data/webtoon_originals_en.csv",
    "zh-hant": "./data/webtoon_originals_zh-hant.csv",
    "th": "./data/webtoon_originals_th.csv",
    "id": "./data/webtoon_originals_id.csv",
    "es": "./data/webtoon_originals_es.csv",
    "fr": "./data/webtoon_originals_fr.csv",
    "de": "./data/webtoon_originals_de.csv",
}

# Caricare i file esistenti in una lista di DataFrame
dataframes = []
for lang, path in file_paths.items():
    try:
        df = pd.read_csv(path)
        df["language"] = lang  # Aggiungere una colonna per indicare la lingua
        dataframes.append(df)
    except FileNotFoundError:
        print(f"File non trovato per la lingua: {lang}")

# Unire tutti i DataFrame in uno solo
merged_df = pd.concat(dataframes, ignore_index=True)

# Salvare il risultato in un nuovo file CSV
output_path = "./data/webtoon_originals_all_languages.csv"
merged_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"File unito creato: {output_path}")


File unito creato: ./data/webtoon_originals_all_languages.csv


In [2]:
# Importare le librerie necessarie
import requests
import pandas as pd
import os

# Funzione per ottenere i dati dai canvas tramite API
def get_canvas_data(lang, rapid_api_key):
    url = "https://webtoon.p.rapidapi.com/canvas/titles/list"
    querystring = {"language": lang}
    
    headers = {
        "x-rapidapi-host": "webtoon.p.rapidapi.com",
        "x-rapidapi-key": rapid_api_key
    }
    
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        response.raise_for_status()  # Verifica se la richiesta è andata a buon fine
        data = response.json()
        titles = data.get("message", {}).get("result", {}).get("titleList", {}).get("titles", [])
        return titles
    except Exception as e:
        print(f"Errore durante la chiamata API per la lingua {lang}: {e}")
        return []

# Funzione per salvare i dati in un CSV
def save_to_csv(titles, lang):
    if not titles:
        print(f"Nessun dato trovato per la lingua {lang}.")
        return
    
    # Creare la cartella per i CSV se non esiste
    output_dir = "/data/canvas_csv"
    os.makedirs(output_dir, exist_ok=True)
    
    # Definire il nome del file CSV
    filename = f"{output_dir}/webtoon_canvas_{lang}.csv"
    
    # Creare un DataFrame e salvarlo in un CSV
    df = pd.DataFrame(titles)
    df.to_csv(filename, index=False, encoding="utf-8")
    
    print(f"File CSV per {lang} salvato come: {filename}")

# Lista delle lingue da cui scaricare i dati dei canvas
langs = ["en", "zh-hant", "th", "id", "es", "fr", "de"]

# Inserire la tua chiave API di RapidAPI qui
rapid_api_key = "df49ecb235msh2daa0d8ec5f5918p1bc1dcjsn1c7de24bf1c9"  # Sostituisci con la tua chiave API

# Scaricare i dati per ogni lingua e salvarli come CSV
for lang in langs:
    print(f"Scaricando dati canvas per la lingua: {lang}")
    titles = get_canvas_data(lang, rapid_api_key)
    save_to_csv(titles, lang)


Scaricando dati canvas per la lingua: en
File CSV per en salvato come: /data/canvas_csv/webtoon_canvas_en.csv
Scaricando dati canvas per la lingua: zh-hant
File CSV per zh-hant salvato come: /data/canvas_csv/webtoon_canvas_zh-hant.csv
Scaricando dati canvas per la lingua: th
File CSV per th salvato come: /data/canvas_csv/webtoon_canvas_th.csv
Scaricando dati canvas per la lingua: id
File CSV per id salvato come: /data/canvas_csv/webtoon_canvas_id.csv
Scaricando dati canvas per la lingua: es
File CSV per es salvato come: /data/canvas_csv/webtoon_canvas_es.csv
Scaricando dati canvas per la lingua: fr
File CSV per fr salvato come: /data/canvas_csv/webtoon_canvas_fr.csv
Scaricando dati canvas per la lingua: de
File CSV per de salvato come: /data/canvas_csv/webtoon_canvas_de.csv


In [ ]:
import pandas as pd

# Percorsi dei file CSV per le lingue (sia originals che canvas)
file_paths = {
    "en": {
        "originals": "/data/webtoon_originals_en.csv",
        "canvas": "/data/webtoon_canvas_en.csv"
    },
    "zh-hant": {
        "originals": "/data/webtoon_originals_zh-hant.csv",
        "canvas": "/data/webtoon_canvas_zh-hant.csv"
    },
    "th": {
        "originals": "/data/webtoon_originals_th.csv",
        "canvas": "/data/webtoon_canvas_th.csv"
    },
    "id": {
        "originals": "/data/webtoon_originals_id.csv",
        "canvas": "/data/webtoon_canvas_id.csv"
    },
    "es": {
        "originals": "/data/webtoon_originals_es.csv",
        "canvas": "/data/webtoon_canvas_es.csv"
    },
    "fr": {
        "originals": "/data/webtoon_originals_fr.csv",
        "canvas": "/data/webtoon_canvas_fr.csv"
    },
    "de": {
        "originals": "/data/webtoon_originals_de.csv",
        "canvas": "/data/webtoon_canvas_de.csv"
    },
}

# Caricare i file per Originals e Canvas in una lista di DataFrame
dataframes = []
for lang, paths in file_paths.items():
    try:
        # Caricare Originals
        df_originals = pd.read_csv(paths["originals"])
        df_originals["language"] = lang
        df_originals["category"] = "originals"
        
        # Caricare Canvas
        df_canvas = pd.read_csv(paths["canvas"])
        df_canvas["language"] = lang
        df_canvas["category"] = "canvas"
        
        # Aggiungere i DataFrame alla lista
        dataframes.append(df_originals)
        dataframes.append(df_canvas)
        
    except FileNotFoundError as e:
        print(f"File non trovato: {e}")

# Unire tutti i DataFrame in uno solo
merged_df = pd.concat(dataframes, ignore_index=True)

# Salvare il risultato in un nuovo file CSV
output_path = " data/webtoon_all_categories.csv"
merged_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"File unito creato: {output_path}")


Errore nella richiesta per search in en: 404
Errore nella richiesta per search in zh-hant: 404
Errore nella richiesta per search in th: 404
Errore nella richiesta per search in id: 404
Errore nella richiesta per search in es: 404
Errore nella richiesta per search in fr: 429
Errore nella richiesta per search in de: 404
Errore nella richiesta per home in en: 429
Errore nella richiesta per home in zh-hant: 429
Errore nella richiesta per home in th: 404
Errore nella richiesta per home in id: 429
Errore nella richiesta per home in es: 429
Errore nella richiesta per home in fr: 404
Errore nella richiesta per home in de: 429
Errore nella richiesta per genres/list in en: 429


FileNotFoundError: [Errno 2] No such file or directory: './data/webtoon_canvas_genres/list_en.csv'